#**Práctica 4b: Problemas de Clasificación, Varios Algoritmos**

Curso: Inteligencia Artificial para Ingenieros

Prof. Carlos Toro N. (carlos.toro.ing@gmail.com)

2022

**Introducción**

En esta práctica continuaremos revisando algoritmos de clasificación y además profundizaremos en temas como hiperparámetros de un modelo y sus ajustes.

**Dentro de los temas que veremos están:**

1. Búsqueda de hiperparámetros
2. Árboles de decisión
3. Varios algoritmos y pipeline completo de clasificación, ejercicios resueltos.
4. Ejercicios propuestos.

## 1. Búsqueda de hiperparámetros

En las prácticas anteriores vimos coomo cargar datos, crear, entrenar y predecir e incluso evaluar el desempeño y capacidad de generalización de modelos predictivos. Aún así, no cambiamos los (hiper) parámetros de los modelos que pueden ser ajustados cuando se crea una instancia de estos. Por ejemplo, para el modelo de k-vecinos más cercanos, `scikit learn` asume por defecto el parámetro: `n_neighbors=5`.

Estos parámetros son llamados **hiperparámetros**, y son usados para controlar el proceso de aprendizaje, por ejemplo el valro k de vecinos en el algoritmo KNN sería un hiperparámetro. Estos son especificados por el usuario, de forma manual o con alguna estrategia de búsqueda (ej. exhaustiva o aleatoria, entre otras), y no se pueden estimar desde los datos. No confundir con los otros parámetros que definen el modelo en si mismo y que se optimizan durante el entrenamiento, por ej. la pendiente y coeficiente de posición son los parámetros que definen un modelo de regresión lineal simple $y = mx + b$.

Para ejemplificar la búsqueda de hiperparámetros, lo haremos con el algoritmo KNN (`KNeighborsClassifier` en sklearn), un conjunto de datos multivariado: el de  [Fisher's iris](https://es.wikipedia.org/wiki/Conjunto_de_datos_flor_iris), de especies de flores iris y la función `GridSearchCV` para encontrar los hiperparámetros óptimos dentro de un conjunto de valores propuestos por el usurio.

**PREVIO:** Importaciones necesarios para la práctica

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# imports de sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

### Importemos y exploremos el dataset



El conjunto de datos de flores iris es un conjunto da datos multivariado y es uno de los más populares a nivel introductorio del machine learning. Este conjunto de datos contiene 150 muestras igualmente distribuidas en tres especies de flores iris: iris setosa, iris virginica, iris versicolor. Cada muestra consiste de 4 atribuos/características/variables/features: la longitud y ancho, en centímetros, del sépalo y pétalo. La tarea es predecir a qué clase pertenece una planta de iris no identificada en base a medicioens del pétalo y sépalo.
<center><img src=https://www.aifunded.es/images/iris.png width="650"></center>

In [ ]:
# url para el dataset Iris
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"

# Asignamos nombres a las columnas del dataset
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class']

# Leemos el dataset
df = pd.read_csv(url, names=names)

CLASS_NAMES = df["Class"].unique()

df.head()

,sepal-length,sepal-width,petal-length,petal-width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
# estará realmente equilibrado el dataset?
df['Class'].value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: Class, dtype: int64

Gráficos de disepersión de las variables e histogramas


In [ ]:
sns.pairplot(df, hue='Class')

### Entrenando el modelo con sus parámetros por defecto
Primero creamos una instancia del modelo y luego lo entrenamos usando una estrategia de validación cruzada para evaluar su desempeño.

In [ ]:
# Datos
X = df.iloc[:,:-1].values
y = df['Class'].values

# Definción modelo y validación cruzada
KNN        = KNeighborsClassifier() # generamos una instancia del modelo con sus parámetros por defecto, en particular, n_neighbors = 5 (default)
CV         = KFold(n_splits = 10, shuffle = True, random_state = 1)# definimos el tipo de estrategia de validación cruzada, KFold en este caso con 10 folds y con reordanmiento aleatorio de los datos originales
cv_results = cross_validate(KNN, X, y, cv = CV, scoring='accuracy', return_train_score=True)# aplicamos la estrategia

# Resultados
print(f"El promedio del accuracy (y su desviación estándar) en los conjuntos de prueba de validación cruzada, con KNN usando 5 vecinos es: "
      f"{cv_results['test_score'].mean():.2f} ({cv_results['test_score'].std():.2f})")

Qué sucede si modificamos el hiperparámetro de número de vecinos del algoritmo?

In [ ]:
N_VECINOS = 3
KNN        = KNeighborsClassifier(n_neighbors = N_VECINOS)
cv_results = cross_validate(KNN, X, y, cv = CV, scoring='accuracy', return_train_score=True)

print(f"El promedio del accuracy (y su desviación estándar) en los conjuntos de prueba de validación cruzada, con KNN usando {N_VECINOS} vecinos es: "
      f"{cv_results['test_score'].mean():.2f} ({cv_results['test_score'].std():.2f})")

Vemos como con cambiar este hiperparámetro manualmente mejoramos el resultado. Automaticemos este proceso de búsqueda y encontremos hiperparámetros óptimos del modelo. Para esto, usaremos la función `GridSearchCV` de  `sklearn`.

Antes de esto, cómo saber que hiperparámetros podemos ajustar?:

1. Mirando la documentación de los modelos en la página de scikit learn o
2. usando el método `get_params` disponible en los modelos de scikit learn para listar los parámetros disponibles y sus valores en una variable tipo diccionario. Igualmente se recomienda revisar la documentación para entender el significado de los parámetros indicados.

Por ejemplo, para el modelo KNN definido anteriormente, los parámetros disponibles son (notar que se despliegan con los valores que se ajusatron más arriba):

In [ ]:
KNN.get_params()

Para implementar la estrategia, especificamos `param_grid =`, el cual es un diccionario que mapea el mombre del hiperparámetro (e.g., `n_neighbors`) a una lista de valores a probar. El modelo con la métrica score más alta se guarda en `.best_estimator_`. Notar que el número de modelos que se probarán será igual al número de combinaciones de valores de hiperparámetros definidos, en el ejemplo de más abajo, se tiene un total de 20x4 combinaciones. Esto hace computacionalmente costosa la estrategia.

In [ ]:
#Dividamos los conjuntos de datos en entrenamiento y test previo a implementar la estrategia
data_train, data_test, target_train, target_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model_knn = KNeighborsClassifier()

# GridSearchCV reemplazará n_neighbors y metric por los valores en param_grid.
PARAM_GRID  = {"n_neighbors": range(1, 21),
               "metric": ["minkowski","euclidean","manhattan","cosine"]}#medida de distancia entre puntos
knn_grid_search = GridSearchCV(model_knn,
                               param_grid= PARAM_GRID,
                               scoring="accuracy",
                               cv=10)# implementa internamente validación cruzada para encontrar la mejor combinación de hiperparámetros
knn_grid_search.fit(data_train, target_train)# ajustamos el modelo usando la estrategia anterior

Exploremos algunos de los resultados obtenidos para las combinaciones de parámetros que se usaron:

In [ ]:
cv_results = pd.DataFrame(knn_grid_search.cv_results_).sort_values("mean_test_score", ascending=False)
cv_results[[
    "param_metric",
    "param_n_neighbors",
    "mean_test_score",
    "std_test_score",
    "rank_test_score"]]

Mejor métrica obtenida qeuda almacenada en el atributo `best_score_`

In [ ]:
print(f"El accuracy al implementar la estrategia es: {knn_grid_search.best_score_:.2f}")# métrica en el mejor caso

Examinemos los mejores parámetros obtenidos en la búsqueda mirando el atributo `best_params_`

In [ ]:
print(f"El mejor conjunto de hiperparámetros (de los que evaluamos) es: {knn_grid_search.best_params_}")

Vemos que en caso de repetirse los resultados óptimos (mean y std) desde la tabla, la estrategia selecciona la primera combinación de hiperparámetros de acuerdo al índice del fold evaluado (n° 62).

Evaluemos la métrica en el conjunto de test que dejamos aparte:

In [ ]:
accuracy_test = knn_grid_search.score(data_test, target_test)
print(f"Accuracy en el conjunto de test: {accuracy_test:.3f}")

La métrica score, en este caso accuracy, estimada en el conjunto de test que dejamos aparte, es cercana al valor óptimo obtenido para la mejor combinación de hiperparámetros y está dentro del rango de los calculados (ver tabla de arriba). Esto implica que el procedimiento de ajuste no causó un sobre-ajuste mayor. Si quisieramos usar el modelo optimo para realizar predicciones en nuevos datos, podemos hacerlo con el método `.predict()` visto en clases anteriores.

**¿será correcto asumir el desempeño de generalización del modelo usando la(s) métrica(s) obtenidas a partir del proceso de búsqueda de hiperparámetros anterior?**

La respuesta es **NO**, yaque estamos buscando los hiperparámetros en el mismo dataset usado para estimar las métricas, el resultado podría estar sesgado a esa selección, aún cuando evaluemos su desempeño en un conjunto de test aparte (como en el ejemplo).

Lo que haremos, será anidar otro proceso de validación cruzada externo para evaluar la capacidad de generalización del modelo y no sobre-estimar su desempeño a la hora de compararlo con otros modelos. Este proceso se conoce como validación cruzada anidada o *nested cross validation* en inglés. La imagen de más abajo resume este procedimiento:


<center><img src=https://hackingmaterials.lbl.gov/automatminer/_images/cv_nested.png width="750" ></center>


[fuente](https://hackingmaterials.lbl.gov/automatminer/advanced.html)

El procedimiento se puede resumir como sigue:

1. Crear una estrategia externa de validación cruzada y una interna para ajuste de hiperparámetros.
2. En cada fold de entrenamiento externo, encontrar el modelo con hiperparámetros óptimos usando una estrategia como búsqueda exhaustiva con `GridSearchCV` u otra, y estimar luego su desempeño para el conjunto de prueba de ese fold externo. Notar que en el ciclo interno también se realiza un procedimiento de validación cruzada.
3. Repetir por cada fold del ciclo externo (outer loop) y estimar las métricas de desempeño de generalización del modelo propuesto.
4. Dado que en cada fold de entrenamiento externo podemos obtener diferentes combinaciones de hiperparámetros óptimos, seleccionar la combinación que más se repita. Si no se repiten, repetir la búsqueda de hiperpárametros (con los mismos ajustes del punto 2) pero en todos los datos disponibles, indicando esta vez como métrica final la obtenida a través de validación cruzada anidada.

 Para el ejemplo de más arriba tendremos:

In [ ]:
#definimos el algoritmo que evaluaremos
model_knn = KNeighborsClassifier()
#definimos la rejilla de hiperparámetros que usaremos con el procedimiento de búsqueda
PARAM_GRID  = {"n_neighbors": range(1, 21),
               "metric": ["minkowski","euclidean","manhattan","cosine"]}
#definimos la estrategia de búsqueda
search      = GridSearchCV(model_knn,
                               param_grid= PARAM_GRID,
                               scoring="accuracy",
                               cv=10)

#definimos la estrategia de validación cruzada
CV         = KFold(n_splits = 5, shuffle = True, random_state = 1)
# aplicamos la estrategia de validación cruzada anidada, con todos los datos disponibles
cv_outer   = cross_validate(search, X, y, cv = CV, scoring='accuracy', return_estimator=True)


Luego, el error de generalización del modelo será:

In [ ]:
cv_outer = pd.DataFrame(cv_outer)
cv_test_scores = cv_outer['test_score']
print("El desempeño de generalización medido con el score accuracy e implementando ajuste de hiperparámtros es: \n"
    f"{cv_test_scores.mean():.3f} ± {cv_test_scores.std():.3f}")

El resultado es compatible con lo encontrado previamente, aunque un poco menor, es más correcto indicar esta métrica y su variabilidad como el desempeño del modelo. Ahora queda indicar cuál es la combinación de hiperparámetros óptima con la que entrenaremos el modelo en todos los datos, veamos los modelos ajustados en cada fold externo (para esto siempre usar `return_estimator = True`):

In [ ]:
for cv_fold, estimator_in_fold in enumerate(cv_outer["estimator"]):
    print(
        f"Mejores hiperparámetros para el modelo en el fold #{cv_fold + 1}:\n"
        f"{estimator_in_fold.best_params_}"
    )

Sería ideal que la selección de hiperparámetros fuera la misma para todos los folds externos, como no es así, repitamos el procedimiento de búsqueda usando todos los datos y configurando los mismos parámetros de búsqueda anteriores:

In [ ]:
search.fit(X, y)# ajustamos los hiperparámetros usando la estrategia de búsqueda definida anteriormente
print(f"El mejor conjunto de hiperparámetros es: {knn_grid_search.best_params_}")

**Conclusión:**

Aún cuando llegamos a la misma conclusión respecto a los hiperparámetros buscados con el procedimiento de validación cruzada simple (`metric = 'cosine` y `n_neighbors= 3`), ahora tenemos una medida de desempeño más correcta y con una indicación de variabilidad de la métrica accuracy (0.967 ± 0.024). Aplicar este procedimiento en futuros análisis.

## 2. Árboles de descisión

Los árboles de decisión son uno de los algoritmos de machine learning más conocidos y usados por su facil interpretación, en este ejemplo, trabajaremos con el mismo conjunto de datos anterior y tendremos cuidado en los hiperparámetros ajustados, yaque, al menos con el de `max_depth`, podemos sobre-ajustar rápidamente el modelo. Partamos por implementar el modelo con ajustes de hiperparámetros básicos.

In [ ]:
# Usemos los datos de clasificación de especies de pingüinos
penguins = pd.read_csv("https://raw.githubusercontent.com/INRIA/scikit-learn-mooc/main/datasets/penguins_classification.csv")

# solo usaremos las clases Adelie y Chinstrap
FEATURES  = ["Culmen Length (mm)", "Culmen Depth (mm)"]
LABELS    = penguins["Species"].unique()
data_p    = penguins[FEATURES].values
target_p  = penguins["Species"].values

# dividamos el conjunto de datos en entrenamiento y test
data_train, data_test, target_train, target_test = train_test_split(data_p, target_p, test_size=0.2, random_state=42)

In [ ]:
# número de datos por especie
penguins["Species"].value_counts()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(max_depth=2, criterion = "gini", random_state=42)
tree_clf.fit(data_train, target_train)

Desempeño del árbol de decisiones entrenado:

In [ ]:
from sklearn.metrics import accuracy_score

print(f"El accuracy en test del árbol de desiciones es: {accuracy_score(target_test, tree_clf.predict(data_test))}\n")
print(f"El accuracy en train del árbol de desiciones es: {accuracy_score(target_train, tree_clf.predict(data_train))}")

Visualicemos el árbol entrenado:

In [ ]:
from sklearn.tree import plot_tree
from sklearn.utils.multiclass import unique_labels # para un orden correcto de las etiquetas como la define sklearn

plt.figure(figsize = [12,7])
plot_tree(tree_clf, feature_names = FEATURES, class_names = unique_labels(target_train, tree_clf.predict(data_train)), filled=True)
plt.title("Árbol de decisión entrenado sin ajuste de hiperparámetros")
plt.show()

En la figura anterior, vemos que para una profundidad 2 del árbol, tenemos una separación de las clases en base a los umbrales de variables estimados en el entrenamiento, en este caso, la métrica gini nos a un indicio de la calidad o impureza de una separación, siendo los valores más pequeños deseables (menor impureza en la separación). La visualización también entrega información de cuántas muestras están siendo clasificadas en cada nodo y por cada clase (lista `value`).   

Apliquemos ahora un procedimiento de búsqueda de hiperparámetros para encontrar el óptimo de `max_depth` y de `criterion`

In [ ]:
#definimos el algoritmo que evaluaremos
model_tree = DecisionTreeClassifier()
#definimos la rejilla de hiperparámetros que usaremos con el procedimiento de búsqueda
PARAM_GRID  = {"max_depth": [1, 3, 5, 7, 9, 11],#profundidad del árbol, qué significa el caso None?
               "criterion": ["gini","entropy"],# criterio de impureza para la división de un nodo
               "min_samples_leaf": range(1, 11,2),# el mínimo número de muestras para estar en un nodo
               "max_leaf_nodes":range(2, 20,2),#los mejores nodos se definen en relación a la reducción relativa de impureza
               "min_samples_split":range(2, 11,2)#el número mínimo de muestras requeridas para dividir un nodo interno
               }
#definimos la estrategia de búsqueda
search_tree = GridSearchCV(model_tree,
                           param_grid= PARAM_GRID,
                           scoring="accuracy",
                           cv=10)
search_tree.fit(data_p, target_p)# ajustamos los hiperparámetros usando la estrategia de búsqueda definida anteriormente
print(f"El mejor conjunto de hiperparámetros es: {search_tree.best_params_}\n")
print(f"El accuracy al implementar la estrategia es: {search_tree.best_score_:.2f}")# métrica en el mejor caso

Entrenemos el modelo en todos los datos con los mejores hiperparámetros encontrados:

In [ ]:
tree_final = DecisionTreeClassifier(max_depth         = search_tree.best_params_['max_depth'],
                                    criterion         = search_tree.best_params_['criterion'],
                                    min_samples_leaf  = search_tree.best_params_['min_samples_leaf'],
                                    max_leaf_nodes    = search_tree.best_params_['max_leaf_nodes'],
                                    min_samples_split = search_tree.best_params_['min_samples_split'],
                                    random_state=42)
tree_final.fit(data_p,target_p)

Visualicemos el árbol para esta configuración encontrada de hiperparámetros óptimos:

In [ ]:
from sklearn.tree import plot_tree

plt.figure(figsize = [16,12])
plot_tree(tree_final,  feature_names = FEATURES, class_names = unique_labels(target_p, tree_clf.predict(data_p)), filled=True)
plt.title("Árbol de decisión entrenado en todos los datos y con hiperparámetros ajustados")
plt.show()

El modelo también nos entrega un atributo para evaluar la importancia de las variables predictoras:

In [ ]:
def plot_feature_importances_tree(model,n_features, feature_names):
    plt.barh(np.arange(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), feature_names)
    plt.xlabel("Importancia de las variables")
    plt.ylabel("Variable")
    plt.ylim(-1, n_features)

plot_feature_importances_tree(tree_final,data_p.shape[1],FEATURES)

**Nota:** solo para efectos de no hacer tan largo del notebook, se omiten algunos pasos vistos anteriormente, cómo estimación de métricas de desempeño, visualizaciones, etc.

Queda como tarea evaluar el modelo desde el punto de vista de la generalización (usando *nested cross validation*)   

**Existirá una forma extra de simplificar el árbol encontrado?**

**SI**, y se conoce como poda (*prunning*), esto recorta o descarta algunas ramas de decisión del árbol antes (*pre-prunning*) o después (*post-prunning*) del entrenamiento, la primera estrategia consiste en definir los hiperparámetros mediante un proceso de búsqueda, como lo hicimos más arriba,la segunda, en analizar el árbol en base a un criterio de complejidad (cost complexity parameter `ccp_alpha` en scikit learn), veamos un ejemplo:

In [ ]:
clf_alpha = DecisionTreeClassifier(random_state=42)
path = clf_alpha.cost_complexity_pruning_path(data_train, target_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities
print(ccp_alphas)

La estrategia de post-poda consistirá en encontrar el parámetro correcto de alpha, analicemos la exactitud par el conjunto de train y test que definimos:

In [ ]:
#Entrenemos modelos para diferentes valores de alpha, fijemos los hiperparámetros encontrados antes
clfs = []
ccp_alphas = list(ccp_alphas[0:len(ccp_alphas)-3])
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=42,
                                 ccp_alpha=ccp_alpha)
    clf.fit(data_train, target_train)
    clfs.append(clf)

#Gráfico de accuracy en train y test
train_scores = [clf.score(data_train, target_train) for clf in clfs]
test_scores = [clf.score(data_test, target_test) for clf in clfs]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha para conjuntos de train y test")
ax.plot(ccp_alphas, train_scores, marker="o", label="train", drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker="o", label="test", drawstyle="steps-post")
ax.legend()
plt.show()

Luego, podemos escoger alpha = 0.01 para entrenar el modelo final y tener una pequeña evaluación de su desempeño, tenemos entonces:

In [ ]:
clf_final = DecisionTreeClassifier(random_state=42,
                                   ccp_alpha=0.01)
clf_final.fit(data_p, target_p)

#árbol con post-poda
plt.figure(figsize = [14,10])
plot_tree(clf_final, feature_names = FEATURES, class_names = unique_labels(target_p, tree_clf.predict(data_p)), filled=True)
plt.title("Árbol de decisión entrenado con post-poda en todos los datos")
plt.show()

In [ ]:
#Desempeño árbol con post-poda
CV         = KFold(n_splits = 10, shuffle = True, random_state = 1)# definimos el tipo de estrategia de validación cruzada, KFold en este caso con 10 folds y con reordanmiento aleatorio de los datos originales
cv_results = cross_validate(DecisionTreeClassifier(random_state=42, ccp_alpha=0.01), data_p, target_p, cv = CV, scoring='accuracy')# aplicamos la estrategia

# Resultados
print(f"El promedio del accuracy (y su desviación estándar) en los conjuntos de prueba de validación cruzada, con árbol de decisión con post poda es: "
      f"{cv_results['test_score'].mean():.2f} ({cv_results['test_score'].std():.2f})")

Así mismo podemos usar también los hiperparámetros ajustados anteriormente antes de la post-poda, cómo cambiará el árbol? **experimentar**.

Matriz de confusión en todos los datos:

In [ ]:
from sklearn.utils.multiclass import unique_labels

target_pred = clf_final.predict(data_p)
UNIQUE_LABELS = unique_labels(target_p, target_pred)# importante!: para obtener las clases en el orden que define sklearn
cm = confusion_matrix(target_p,target_pred,labels = UNIQUE_LABELS)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=UNIQUE_LABELS)
disp.plot()
plt.title("Matriz de confusión en datos disponibles")
plt.show()

Por último, podemos estimar la probabilidad con que se predice la clase de una muestra de datos:

In [ ]:
sample          = data_p[0:1,:]# ejemplificamos con una muestra tomada desde el dataset original
y_pred_proba    = clf_final.predict_proba(sample)
y_proba_class_0 = pd.Series(y_pred_proba[0], index=clf_final.classes_)
print(f"Probabilidades de la muestra {sample} de pertenecer a una clase:\n{y_proba_class_0}")
y_proba_class_0.plot.bar()
plt.ylabel("Probabilidad")
_ = plt.title("Probabilidad de pertenecer a la clase de un pungüino")

##Ejercicios extra

**1.** En el ejemplo de árboles de decisión (ejemplo usando ajuste de hiperparámetros), usar validación cruzada anidada para entregar una estimación de la capacidad de generalización del modelo en base a una métrica de desempeño definida.

**2.** Implementar un ejemplo para el conjunto de datos de especies de pingüinos comparando el desempeño de los modelos de: i) regresión logística, ii) k-vecinos más cercanos, iii) árboles de decisión y iv) máquinas de soporte vectorial.
- Implementar validación cruzada anidada con búsqueda de al menos un hiperparámetro por modelo en el reporte de sus resultados.
- ¿qué modelo seleccionaría para estos datos en base a este análisis?

- Mostrar la matriz de confusión para el modelo seleccionado y evaluado en todos los datos.

**4.** Nombre las ventajas y desventajas del algoritmo de árboles de decisión.

**5.** Investigue para qué algoritmos de clasificación es relevante la estandarización o escalado de las variables predictoras previo al entrenamiento y programe un ejemplo donde se aprecie el efecto de realizar o no este pre-procesamiento.

**6.** Reproducir el ejemplo mostrado en la documentación de scikit learn para clasificación de dígitos escritos a mano: [descrito aquí](https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html), pero incluyendo una estrategia de validación cruzada anidada y procedimiento de búsqueda para selección de al menos dos hiperparámetros del algoritmo.

## Referencias

**1.** Algunos de los contenidos fueron adaptados del curso "Machine Learning in Python with scikit-learn" de INRIA (National Institute for Research in Digital Science and Technology, Francia), lo pueden tomar gratuitamente acá: https://www.fun-mooc.fr/en/courses/machine-learning-python-scikit-learn/

**2.** Para profundizar en la búsqueda de hiperparámetros: [enlace1](https://www.youtube.com/watch?v=YAfS8-BXp8Q), [enlace2](https://www.youtube.com/watch?v=WHcEg4HXsH4), [enlace3](https://www.youtube.com/watch?v=tIO8zPCdi58).

**3.** Algoritmo K-Vecinos mas Cercanos: [enlace](https://python-course.eu/machine-learning/k-nearest-neighbor-classifier-with-sklearn.php)

**4.** Algoritmo de máquinas de soporte vectorial, importancia del estandarizado de los datos: [enlace](https://winder.ai/support-vector-machines/).
**5.** Algoritmo de máquina de soporte vectoria, ejemplo clasificación de imágenes de rostros: [enlace](https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.07-Support-Vector-Machines.ipynb#scrollTo=JO0LrAJlIMEH).

**6.** Algoritmo de máquina de soporte vectorial con ajuste de hiperparámetros: [enlace](https://www.youtube.com/watch?v=b4qQzVPWkZQ).

**7.** Bosques aleatorios: [enlace](https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.08-Random-Forests.ipynb#scrollTo=yKndK0APLLj3).
